<a href="https://colab.research.google.com/github/kimgoinghard/study-machine-learning/blob/main/study_torch_4_multi_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

변수가 1개인 선형 회귀를 단순선형회귀라고 했다. 변수가 2개 이상인 경우는 다중선형회귀라고 한다. 

#H(x) = w1x1 + w2x2 + w3x3 + b

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1) #랜덤시드 고정

In [3]:
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [24]:
#매개변수 생성 및 초기화
w1 = torch.zeros(1, requires_grad = True)
w2 = torch.zeros(1, requires_grad = True)
w3 = torch.zeros(1, requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [25]:
optimizer = optim.SGD([w1,w2,w3,b], lr = 1e-5) 

In [26]:
n_epochs = 1000
for epoch in range(n_epochs + 1):
  #hypo라고 썼지만 y_hat과 같은 의미.
  hypo = x1_train*w1 + x2_train*w2 + x3_train*w3 + b

  cost = torch.mean((hypo - y_train)**2)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%100 ==0:
    print('Epoch : {0}, w1 : {1}, w2 : {2} , w3 : {3}, b : {4}'.format(epoch, w1.item(), w2.item(), w3.item(), b ))

Epoch : 0, w1 : 0.29401201009750366, w2 : 0.2935999929904938 , w3 : 0.2973800003528595, b : tensor([0.0034], requires_grad=True)
Epoch : 100, w1 : 0.6735064387321472, w2 : 0.6609678864479065 , w3 : 0.6762318015098572, b : tensor([0.0079], requires_grad=True)
Epoch : 200, w1 : 0.6789457201957703, w2 : 0.6549615859985352 , w3 : 0.6767846345901489, b : tensor([0.0081], requires_grad=True)
Epoch : 300, w1 : 0.6842508316040039, w2 : 0.6491218209266663 , w3 : 0.6773056983947754, b : tensor([0.0082], requires_grad=True)
Epoch : 400, w1 : 0.6894257068634033, w2 : 0.643443763256073 , w3 : 0.6777958869934082, b : tensor([0.0084], requires_grad=True)
Epoch : 500, w1 : 0.6944737434387207, w2 : 0.6379231810569763 , w3 : 0.6782558560371399, b : tensor([0.0085], requires_grad=True)
Epoch : 600, w1 : 0.69939786195755, w2 : 0.6325559616088867 , w3 : 0.678686797618866, b : tensor([0.0087], requires_grad=True)
Epoch : 700, w1 : 0.7042017579078674, w2 : 0.6273379325866699 , w3 : 0.679089367389679, b : ten

위의 경우에서 x가 3개가 아니라 1000개라고 생각해보자. 그 경우엔 x1~x1000, w1~w1000 매개변수만 2000개를 선언하고 곱해지는 항도 개수가 너무 많아져 상당히 비효율적이라고 할 수 있다. 

#그래서! 행렬곱셉을 사용한다. 

In [27]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])
#x_train을 (5,3)행렬로 생성했다,

In [29]:
w = torch.zeros((3,1), requires_grad = True) 
b = torch.zeros((5,1), requires_grad = True)

w행렬의 차원을 보면 (3,1)인데 이것이 포인트다. 행렬 곱셉에선 앞 행렬의 열과 뒷 행렬의 행이 같아야 한다. 현재 x_train은 (5,3)이다. w는 (3,1)로 곱셈이 가능하다!

In [32]:
hypo = x_train.matmul(w) + b

In [34]:
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5) 

nb_epochs = 20
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해진다.
    hypo = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypo - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad() #0으로 초기화
    cost.backward() #역전파로 미분계수 계산
    optimizer.step() #미분계수 업데이트

    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypo.squeeze().detach(), cost.item() #hypo 는 (3,1)인데 squeeze해주어 (3,)으로 바뀐다. 
    )) #detach()는 이후의 연산이 추적되는 것을 방지한다.

Epoch    0/20 hypothesis: tensor([154.0543, 185.1140, 175.7470, 198.6150, 141.2166]) Cost: 5.954494
Epoch    1/20 hypothesis: tensor([154.0546, 185.1143, 175.7486, 198.6150, 141.2171]) Cost: 5.951927
Epoch    2/20 hypothesis: tensor([154.0547, 185.1143, 175.7500, 198.6147, 141.2173]) Cost: 5.949370
Epoch    3/20 hypothesis: tensor([154.0546, 185.1142, 175.7512, 198.6143, 141.2175]) Cost: 5.946794
Epoch    4/20 hypothesis: tensor([154.0546, 185.1140, 175.7523, 198.6138, 141.2177]) Cost: 5.944210
Epoch    5/20 hypothesis: tensor([154.0545, 185.1137, 175.7535, 198.6133, 141.2178]) Cost: 5.941589
Epoch    6/20 hypothesis: tensor([154.0543, 185.1135, 175.7545, 198.6127, 141.2178]) Cost: 5.939054
Epoch    7/20 hypothesis: tensor([154.0542, 185.1132, 175.7556, 198.6122, 141.2179]) Cost: 5.936491
Epoch    8/20 hypothesis: tensor([154.0541, 185.1129, 175.7567, 198.6116, 141.2180]) Cost: 5.933901
Epoch    9/20 hypothesis: tensor([154.0539, 185.1126, 175.7578, 198.6110, 141.2180]) Cost: 5.931320
